In [1]:
import pandas as pd
import numpy as np
import joblib
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from xgboost import XGBClassifier

# ==============================
# PATH SETUP
# ==============================
PROJECT_ROOT = r"C:\Users\sande\Desktop\Python\Credit-Card-Fraud-Detection"
ARTIFACTS_DIR = os.path.join(PROJECT_ROOT, "artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# ==============================
# 1. LOAD DATA
# ==============================
data = pd.read_csv(
    r"C:\Users\sande\Desktop\Python\Credit-Card-Fraud-Detection\data\creditcard.csv"
)

X = data.drop("Class", axis=1)
y = data["Class"]

feature_names = X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ==============================
# 2. SCALE FEATURES
# ==============================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ==============================
# 3. HANDLE IMBALANCE
# ==============================
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    scale_pos_weight=neg / pos,
    eval_metric="logloss",
    random_state=42
)

model.fit(X_train_scaled, y_train)

# ==============================
# 4. COST-BASED THRESHOLD
# ==============================
probs = model.predict_proba(X_test_scaled)[:, 1]

COST_FN = 100
COST_FP = 5

precision, recall, thresholds = precision_recall_curve(y_test, probs)

best_threshold = 0.5
min_cost = float("inf")

for t in thresholds:
    preds = (probs >= t).astype(int)
    fp = ((preds == 1) & (y_test == 0)).sum()
    fn = ((preds == 0) & (y_test == 1)).sum()
    cost = fp * COST_FP + fn * COST_FN

    if cost < min_cost:
        min_cost = cost
        best_threshold = float(t)

# ==============================
# 5. DRIFT BASELINE (CRITICAL)
# ==============================
baseline_stats = {}

for i, feature in enumerate(feature_names):
    baseline_stats[feature] = {
        "mean": float(np.mean(X_train_scaled[:, i])),
        "std": float(np.std(X_train_scaled[:, i]))
    }

# ==============================
# 6. SAVE ARTIFACTS
# ==============================
joblib.dump(model, os.path.join(ARTIFACTS_DIR, "model.pkl"))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, "scaler.pkl"))

with open(os.path.join(ARTIFACTS_DIR, "threshold.json"), "w") as f:
    json.dump({"threshold": best_threshold}, f)

with open(os.path.join(ARTIFACTS_DIR, "baseline_stats.json"), "w") as f:
    json.dump(baseline_stats, f)

print("✅ All artifacts saved to:", ARTIFACTS_DIR)

✅ All artifacts saved to: C:\Users\sande\Desktop\Python\Credit-Card-Fraud-Detection\artifacts


In [ ]:
# import pandas as pd
# import numpy as np
# import joblib
# import json

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import precision_recall_curve
# from xgboost import XGBClassifier

# import os
# PROJECT_ROOT = r"C:\Users\sande\Desktop\Python\Credit-Card-Fraud-Detection"
# ARTIFACTS_DIR = os.path.join(PROJECT_ROOT, "artifacts")
# os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# # 1. Load data
# data = pd.read_csv("C:/Users/sande/Desktop/Python/Credit-Card-Fraud-Detection/data/creditcard.csv")

# X = data.drop("Class", axis=1)
# y = data["Class"]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=y, random_state=42
# )

# # 2. Scale (keep it simple)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # 3. Handle imbalance
# neg = (y_train == 0).sum()
# pos = (y_train == 1).sum()

# model = XGBClassifier(
#     n_estimators=300,
#     max_depth=5,
#     learning_rate=0.05,
#     scale_pos_weight=neg / pos,
#     eval_metric="logloss",
#     random_state=42
# )

# model.fit(X_train, y_train)

# # 4. Threshold selection (simple cost-based)
# probs = model.predict_proba(X_test)[:, 1]

# COST_FN = 100   # fraud loss
# COST_FP = 5     # customer friction

# precision, recall, thresholds = precision_recall_curve(y_test, probs)

# best_threshold = 0.5
# min_cost = float("inf")

# for t in thresholds:
#     preds = (probs >= t).astype(int)
#     fp = ((preds == 1) & (y_test == 0)).sum()
#     fn = ((preds == 0) & (y_test == 1)).sum()
#     cost = fp * COST_FP + fn * COST_FN

#     if cost < min_cost:
#         min_cost = cost
#         best_threshold = float(t)

# # 5. Save artifacts
# joblib.dump(model, os.path.join(ARTIFACTS_DIR, "model.pkl"))
# joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, "scaler.pkl"))

# with open(os.path.join(ARTIFACTS_DIR, "threshold.json"), "w") as f:
#     json.dump({"threshold": best_threshold}, f)

# print("Artifacts saved to:", ARTIFACTS_DIR)


Artifacts saved to: C:\Users\sande\Desktop\Python\Credit-Card-Fraud-Detection\artifacts
